In [29]:
import pymysql
from functools import partial

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)

connection = pymysql.connect(
    host='localhost',
    user='megadados',
    password='12345',
    database='birdbook')
cursor = connection.cursor()
db = partial(run_db_query, connection)

```SQL
USE birdbook;

DROP PROCEDURE IF EXISTS adiciona_post;

DELIMITER //
CREATE PROCEDURE adiciona_post(IN novo_titulo VARCHAR(80), IN novo_texto VARCHAR(80))
BEGIN
    INSERT INTO POST (titulo, texto) VALUES (novo_titulo, novo_texto);
END//
DELIMITER ;```

In [129]:
def parser_post(texto):
    lista_texto = texto.split(' ')
    lista_passaros = []
    lista_citados = []
    for item in lista_texto:
        if item[0] == '#':
            lista_passaros.append(item[1:])
        if item[0] == '@':
            lista_citados.append(item[1:])
    return {"#": lista_passaros, "@": lista_citados}

def insere_post(cursor, novotitulo, novotexto, novaURL_foto):
    cursor.execute('''
        INSERT INTO POST(titulo, texto, URL_foto)
        VALUES(%s,%s,%s)
    ''', (novotitulo, novotexto, novaURL_foto))
    cursor.execute('''
        SELECT LAST_INSERT_ID()
            FROM POST
     ''')
    return cursor.fetchone()[0]

def cria_tags(cursor,dici_tags, id_post):
    resp = {}
    resp_pes = {}
    if '#' in dici_tags:
        for item in dici_tags["#"]:
            resp[item] = 0
        for item in dici_tags["#"]:
            print(item)
            cursor.execute('''
                SELECT COUNT(tag_PASSARO) FROM PASSARO
                WHERE tag_PASSARO = %s; 
            ''',(item))
            resp[item] += cursor.fetchone()[0]
        for pas in resp:
            if resp[pas] > 0:
                cursor.execute('''
                INSERT INTO TAG_PASSARO_POST (idPost,tag_PASSARO)
                VALUES(%s,%s)
                ''',(id_post,pas))
                
    if '@' in dici_tags:
        for item in dici_tags["@"]:
            resp_pes[item] = 0
        for item in dici_tags["@"]:
            print(item)
            cursor.execute('''
                SELECT COUNT(username) FROM USUARIO
                WHERE username = %s; 
            ''',(item))
            resp_pes[item] += cursor.fetchone()[0]
        for usu in resp_pes:
            if resp_pes[usu] > 0:
                cursor.execute('''
                INSERT INTO TAG_USUARIO_POST (idPost,username)
                VALUES(%s,%s)
                ''',(id_post,usu))

In [126]:
texto = "@pessoa #piri #pas #teste"
idpost = insere_post(cursor,"manu",texto,"Null")
dici_tags = parser_post(texto)
cria_tags(cursor,dici_tags, idpost)
print(idpost)

piri
pas
teste
pessoa
conts usuario 1
conts usuario 1
conts usuario 1
31


In [124]:
db('''
INSERT INTO CIDADE (cidade) VALUES('sao paulo')''')
db('''
INSERT INTO USUARIO (username, email, nome, cidade) VALUES('pessoa','pesso@pessoa','pessoainha mm','sao paulo')''')

Executando query:
Executando query:


In [123]:
db('''DESCRIBE CIDADE''')

Executando query:
('cidade', 'varchar(45)', 'NO', 'PRI', None, '')


In [96]:
print(teste)

((1,),)


In [127]:
db('SELECT * FROM TAG_USUARIO_POST')

Executando query:
('pessoa', 31)


In [44]:
db('ROLLBACK')

Executando query:


In [28]:
connection.close()

In [128]:
teste

{'piri': 1, 'pas': 1, 'teste': 1}